<a href="https://colab.research.google.com/github/eunnjji/MultiCampus-EmpAcademy/blob/main/project2/SYH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SYH

- 팀명: SYH : super youth hero
- 주제: 기계 시설물 분야 AI 학습용 데이터를 활용한 기계 시설물의 고장 여부 예측 모델 개발
- 여수진, 성은지, 홍승표
- [참고](https://www.aidatahackathon.com/)

## 절차
1. 데이터 정제
1. 모델 생성
1. 모델 예측

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# libarary import
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn as skl
import pickle

# seed setting
np.random.seed(42)
tf.random.set_seed(42)

BASE_DIR = "/content/drive/MyDrive/"
DATA_DIR = '/content/drive/MyDrive/project/daejeon_data'

지도 학습 문제, 회귀 문제,   
다중 분류 (소프트맥스 회귀)  
CNN 계열 모델 활용 

# 1. 데이터 정제 및 분석


In [ ]:
%cd '/content/drive/MyDrive'
os.getcwd()

/content/drive/MyDrive


'/content/drive/MyDrive'

In [ ]:
classSet = ("bearing", "belt", "misalignment", "unbalance")

def DatadirLoad(dirname):
    csv_op_dict = {classSet[0]:[], classSet[1]:[], classSet[2]:[],classSet[3]:[]}
    csv_no_dict = {classSet[0]:[], classSet[1]:[], classSet[2]:[],classSet[3]:[]}

    Save_op_Name, Save_no_Name = 'Data_op_dir.pickle','Data_no_dir.pickle'
    Save_op_path = os.path.join(BASE_DIR,Save_op_Name)
    Save_no_path = os.path.join(BASE_DIR,Save_no_Name)

    if os.path.isdir(Save_op_path):
        with open(Save_op_path, 'rb') as fr:
            csv_op_dict = picle.load(fr)
    if os.path.isdir(Save_no_path):
        with open(Save_no_path, 'rb') as fr:
            csv_no_dict = picle.load(fr)
        return csv_op_dict,csv_no_dict

    dirpath = os.path.join(DATA_DIR,dirname)
    if os.path.isdir(dirpath):
        for d in os.listdir(dirpath):
            d_path = dirpath+'/'+d
            if os.path.isdir(d_path):
                Error_Class_Lable,kw_Name,*_ = d.split('_')
                print(Error_Class_Lable,kw_Name)
                data_dir_list = [ x for x in os.listdir(d_path) if len(x)<2]
                data_dir_list.sort()
                #print(data_dir_list)
                if len(data_dir_list) > 0:
                    for d2 in data_dir_list:
                        d2_path = os.path.join(d_path,d2)

                        # op-csv file # 정상 작동
                        d2_op_path = os.path.join(d2_path,'4.op-csv')
                        d2_op_data = [x for x in os.listdir(d2_op_path) if x.endswith('.csv')]
                        op_csv_path = os.path.join(d2_op_path,d2_op_data[0])
                        csv_op_dict[Error_Class_Lable].append(op_csv_path)
                        with open(Save_op_path,'wb') as fw:
                            pickle.dump(csv_op_dict,fw)
                        # no-csv file # 고장 발생
                        d2_no_path = os.path.join(d2_path,'5.no-csv')
                        d2_no_data = [x for x in os.listdir(d2_no_path) if x.endswith('.csv')]
                        no_csv_path = os.path.join(d2_no_path,d2_no_data[0])
                        csv_no_dict[Error_Class_Lable].append(no_csv_path)
                        with open(Save_no_path,'wb') as fw:
                            pickle.dump(csv_no_dict,fw)

    return csv_op_dict, csv_no_dict

In [ ]:
class_name = {'bearing':0, 'belt':1, 'misalignment':2, 'normal':3, 'unbalance':4}

In [ ]:
def read_csv_dir(path, max_row_size):
    t_pd = None
    update_row_size = max_row_size
    y = None
    if os.path.isfile(path):
        data_Name = os.path.basename(path)
        dir_name,Ekw,dir_num,opno  = os.path.dirname(path).split('/')[6:]
        Error_Class_Lable,kw_Name,*_ = Ekw.split('_')      
        opno = 0 if opno[0] == '4' else 1
        # 정상 : False(0), 고장 : True(1)

        if dir_name[0] == 'c': #current(전류)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(15))
            t_pd.drop(3, axis=1, inplace=True)
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = class_name[Error_Class_Lable] if opno == 1 else class_name['normal']
            y = t_pd['target']
            t_pd.drop(['target'], axis=1, inplace=True)

        elif dir_name[0] == 'k': # kimm(유선진동)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(4))
            t_pd.drop(1, axis=1, inplace=True)
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = class_name[Error_Class_Lable] if opno == 1 else class_name['normal']
            y = t_pd['target']
            t_pd.drop(['target'], axis=1, inplace=True)

        elif dir_name[0] == 'v': #vibration(무선진동)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(4))
            t_pd.drop(1, axis=1, inplace=True)
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = class_name[Error_Class_Lable] if opno == 1 else class_name['normal']
            y = t_pd['target']
            t_pd.drop(['target'], axis=1, inplace=True)
            if (t_pd.values.shape[0]) > max_row_size:
                update_row_size = t_pd.values.shape[0]
            elif (t_pd.values.shape[0]) < max_row_size:
                update_row_size = 6000
                tmp = np.zeros((int(update_row_size - t_pd.values.shape[0]),int(t_pd.values.shape[1])))
                #y_tmp = np.zeros(y.values.shape) # 이렇게 될 경우 normal 데이터에 bearing 데이터가 함께 존재하게 됨
                y_tmp = y.values
                new_t = np.vstack([t_pd.values,tmp])
                new_y = np.append(y,y_tmp,axis=0)
                return new_t.tolist(), update_row_size, new_y.tolist()

        else: 
            print('error')
            print('dir_category : {}, lable : {}, op: {}'.format(dir_name, Error_Class_Lable,opno))
            return 0
    return t_pd.values.tolist(), update_row_size, y.values.tolist()
    

In [ ]:
# dataset 생성
def makeDataset(op_dict, no_dict, category_num, shape_op = 0):
    """
    input: 정상data의 path list, 오류 data의 path list
    output: numpy의 ndarray 타입 형태로 dataset 반환
    category_num:
    0은 current 데이터
    1은 유선 진동 데이터 
    2는 무선 진동 데이터 
    shape_op :
     0이면 (38200, 60 , 5(특성 수)) 형태, 
     1이면 (38200(data개수 * row/60한 값), 5(특성 수), 60)
    """
    tmp = []
    max_row_size = 0
    y_tmp = []
    Y = None

    category_name = None
    if category_num == 0:
        category_name = 'current'
    elif category_num == 1:
        category_name = 'kimm'
    else:category_name = 'vibration'

    save_x_Name, save_y_Name = category_name+'_x.pickle',category_name+'_y.pickle'
    save_x_path = os.path.join(BASE_DIR,save_x_Name)
    save_y_path = os.path.join(BASE_DIR,save_y_Name)



    for k,v in op_dict.items():
        for p in v:
            data, max, y = read_csv_dir(p, max_row_size)
            tmp.append(data)
            y_tmp.append(y)
            max_row_size = max
    for k,v in no_dict.items():
        for p in v:
            data, max, y = read_csv_dir(p, max_row_size)
            tmp.append(data)
            y_tmp.append(y)
            max_row_size = max
    
    if shape_op == 0:
        dataset = np.stack(tmp)
        feature_num = 4 if category_num == 0 else 2
        Y = np.stack(y_tmp)
        shape = (-1,60,feature_num) 
        y_shape = (-1,60)

    else:
        dataset = np.stack(tmp,axis=2)
        feature_num = 4 if category_num == 0 else 2
        dataset = dataset.reshape((-1,feature_num,60)).copy()
        Y = np.stack(y_tmp)
        Y = Y.reshape((-1,60)).copy()
        shape = (-1,feature_num,60)
        y_shape = (-1,60)
    with open(save_x_path,'wb') as fw:
        pickle.dump(dataset.reshape(shape),fw)
    with open(save_y_path,'wb') as fw:
        pickle.dump(Y.reshape(y_shape),fw)

    return  dataset.reshape(shape), Y.reshape(y_shape)

In [ ]:
def load_dataset(category_num=0):
    """
    0: current dataset
    1: kimm dataset
    2: vibration dataset
    """
    category_name = None
    if category_num == 0:
        category_name = 'current'
    elif category_num == 1:
        category_name = 'kimm'
    else:category_name = 'vibration'

    load_x_Name, load_y_Name = category_name+'_x.pickle',category_name+'_y.pickle'
    load_x_path = os.path.join(BASE_DIR,load_x_Name) #ex. /content/drive/MyDrive/current_x.pickle
    load_y_path = os.path.join(BASE_DIR,load_y_Name) 

    if os.path.isfile(load_x_path):
        print('exist')
        with open(load_x_path, 'rb') as fr:
            X = pickle.load(fr)
        if os.path.isfile(load_y_path):
            print('exist')
            with open(load_y_path, 'rb') as fr:
                y = pickle.load(fr)
            return X,y
    else:
        print('not exist')
        if category_num == 0:
            current_op_csv,current_no_csv = DatadirLoad("current(전류)")
            current_x , current_y = makeDataset(current_op_csv,current_no_csv,0,shape_op=0)
            return current_x,current_y
        elif category_num == 1:
            kimm_op_csv, kimm_no_csv = DatadirLoad("kimm(유선진동)")
            kimm_x, kimm_y = makeDataset(kimm_op_csv,kimm_no_csv, 1)
            return kimm_x,kimm_y
        else:
            vibration_op_csv, vibration_no_csv = DatadirLoad("vibration(무선진동)")
            vibration_x, vibration_y = makeDataset(vibration_op_csv, vibration_no_csv,2)
            return vibration_x, vibration_y


In [ ]:
pred_name = ['bearing', 'belt', 'misalignment','normal', 'unbalance']

In [ ]:
current_x,current_y = load_dataset(0) # current

exist
exist


In [ ]:
current_x.shape,current_y.shape

((38200, 60, 4), (38200, 60))

In [ ]:
kimm_x,kimm_y = load_dataset(1) # kimm

exist
exist


In [ ]:
kimm_x.shape, kimm_y.shape

((76800, 60, 2), (76800, 60))

In [ ]:
vibration_x, vibration_y = load_dataset(2) # vibr

exist
exist


In [ ]:
vibration_x.shape, vibration_y.shape

((38400, 60, 2), (38400, 60))

# 2. 모델 생성

## 전류 데이터

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(current_x,current_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train.shape, y_train.shape

((22920, 60, 4), (22920,))

### CNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(current_x,current_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

(22920, 60, 4, 1)

In [ ]:
# 각 특성 값의 범위를 0~1로 조정
X_train[:,:,3:] = X_train[:,:,3:]/100
X_valid[:,:,3:] = X_valid[:,:,3:]/100
X_test[:,:,3:] = X_test[:,:,3:]/100

In [ ]:
from functools import partial

cnn_model = keras.models.Sequential([
    keras.layers.Conv2D(64,7,activation="relu",padding="same",input_shape=(60,4,1)),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=5, activation='softmax')
])

In [ ]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 4, 64)         3200      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 4, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 4, 128)        73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 4, 128)        147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 2, 128)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 2, 256)        295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 2, 256)        5

In [ ]:
cnn_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
history = cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
717/717 [==============================] - 15s 9ms/step - loss: 0.7194 - accuracy: 0.6643 - val_loss: 0.0312 - val_accuracy: 0.9937
Epoch 2/10
717/717 [==============================] - 6s 9ms/step - loss: 0.1295 - accuracy: 0.9712 - val_loss: 0.0131 - val_accuracy: 0.9973
Epoch 3/10
717/717 [==============================] - 6s 9ms/step - loss: 0.2290 - accuracy: 0.9852 - val_loss: 0.0138 - val_accuracy: 0.9970
Epoch 4/10
717/717 [==============================] - 6s 9ms/step - loss: 0.0580 - accuracy: 0.9886 - val_loss: 0.0122 - val_accuracy: 0.9973
Epoch 5/10
717/717 [==============================] - 6s 9ms/step - loss: 0.0229 - accuracy: 0.9939 - val_loss: 0.0084 - val_accuracy: 0.9973
Epoch 6/10
717/717 [==============================] - 6s 9ms/step - loss: 0.0606 - accuracy: 0.9878 - val_loss: 0.0086 - val_accuracy: 0.9973
Epoch 7/10
717/717 [==============================] - 6s 9ms/step - loss: 0.8832 - accuracy: 0.9899 - val_loss: 0.0086 - val_accuracy: 0.9973
Epoch

### ResNet 34

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(current_x,current_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train[:,:,3:] = X_train[:,:,3:]/100
X_valid[:,:,3:] = X_valid[:,:,3:]/100
X_test[:,:,3:] = X_test[:,:,3:]/100

In [ ]:
X_train.shape

(22920, 60, 4, 1)

In [ ]:
# 잔차유닛
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()]
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
current_resnet_model = keras.models.Sequential()
current_resnet_model.add(keras.layers.Conv2D(64,7,strides=1,input_shape=[60,4,1],padding="same",use_bias=False)) # current의 경우 strides를 1로 해야 함
current_resnet_model.add(keras.layers.BatchNormalization())
current_resnet_model.add(keras.layers.Activation("relu"))
current_resnet_model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="same"))
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides = 1 if filters == prev_filters else 2
    current_resnet_model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
current_resnet_model.add(keras.layers.GlobalAvgPool2D())
current_resnet_model.add(keras.layers.Flatten())
current_resnet_model.add(keras.layers.Dense(5,activation="softmax"))

In [ ]:
current_resnet_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
current_resnet_model.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
717/717 [==============================] - 47s 56ms/step - loss: 0.8921 - accuracy: 0.6209 - val_loss: 98.0629 - val_accuracy: 0.4674
Epoch 2/10
717/717 [==============================] - 39s 54ms/step - loss: 0.5593 - accuracy: 0.7496 - val_loss: 5.4263 - val_accuracy: 0.5024
Epoch 3/10
717/717 [==============================] - 39s 55ms/step - loss: 0.3987 - accuracy: 0.8021 - val_loss: 27.6360 - val_accuracy: 0.5020
Epoch 4/10
717/717 [==============================] - 39s 55ms/step - loss: 0.3319 - accuracy: 0.8578 - val_loss: 18.5321 - val_accuracy: 0.5312
Epoch 5/10
717/717 [==============================] - 39s 55ms/step - loss: 0.2940 - accuracy: 0.8833 - val_loss: 58.3590 - val_accuracy: 0.5018
Epoch 6/10
717/717 [==============================] - 39s 55ms/step - loss: 0.2555 - accuracy: 0.9006 - val_loss: 119.1620 - val_accuracy: 0.6071
Epoch 7/10
717/717 [==============================] - 39s 55ms/step - loss: 0.1942 - accuracy: 0.9309 - val_loss: 28.3440 - val_ac

## 유선진동

In [ ]:
pred_name = ['bearing', 'belt', 'misalignment','normal', 'unbalance']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(kimm_x,kimm_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

### ResNet-34 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(kimm_x,kimm_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

(46080, 60, 2, 1)

In [ ]:
# 각 특성 값의 범위를 0~1로 조정
X_train[:,:,1:] = X_train[:,:,1:]/100
X_valid[:,:,1:] = X_valid[:,:,1:]/100
X_test[:,:,1:] = X_test[:,:,1:]/100

In [ ]:
# 잔차유닛
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()]
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
kimm_resnet_model = keras.models.Sequential()
kimm_resnet_model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[60,2,1],padding="same",use_bias=False))
kimm_resnet_model.add(keras.layers.BatchNormalization())
kimm_resnet_model.add(keras.layers.Activation("relu"))
kimm_resnet_model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="same"))
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides = 1 if filters == prev_filters else 2
    kimm_resnet_model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
kimm_resnet_model.add(keras.layers.GlobalAvgPool2D())
kimm_resnet_model.add(keras.layers.Flatten())
kimm_resnet_model.add(keras.layers.Dense(5,activation="softmax"))

In [ ]:
kimm_resnet_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
kimm_resnet_model.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1440/1440 [==============================] - 77s 49ms/step - loss: 1.2793 - accuracy: 0.5314 - val_loss: 0.9604 - val_accuracy: 0.6442
Epoch 2/10
1440/1440 [==============================] - 69s 48ms/step - loss: 0.8746 - accuracy: 0.6466 - val_loss: 0.9472 - val_accuracy: 0.6981
Epoch 3/10
1440/1440 [==============================] - 69s 48ms/step - loss: 0.6787 - accuracy: 0.7134 - val_loss: 2.1146 - val_accuracy: 0.5268
Epoch 4/10
1440/1440 [==============================] - 69s 48ms/step - loss: 0.5793 - accuracy: 0.7541 - val_loss: 1.4651 - val_accuracy: 0.6014
Epoch 5/10
1440/1440 [==============================] - 69s 48ms/step - loss: 0.5085 - accuracy: 0.7799 - val_loss: 1.6573 - val_accuracy: 0.6134
Epoch 6/10
1440/1440 [==============================] - 69s 48ms/step - loss: 0.4448 - accuracy: 0.8014 - val_loss: 1.6234 - val_accuracy: 0.6847
Epoch 7/10
1440/1440 [==============================] - 69s 48ms/step - loss: 0.4057 - accuracy: 0.8203 - val_loss: 1.6951 -

## 무선진동

In [ ]:
vibration_x.shape, vibration_y.shape

((38400, 60, 2), (38400, 60))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vibration_x,vibration_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

### Resnet34

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vibration_x,vibration_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_train[:,:,1:] = X_train[:,:,1:]/100
X_valid[:,:,1:] = X_valid[:,:,1:]/100
X_test[:,:,1:] = X_test[:,:,1:]/100

In [ ]:
# 잔차유닛
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()]
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
vibration_resnet_model = keras.models.Sequential()
vibration_resnet_model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[60,2,1],padding="same",use_bias=False))
vibration_resnet_model.add(keras.layers.BatchNormalization())
vibration_resnet_model.add(keras.layers.Activation("relu"))
vibration_resnet_model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="same"))
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides = 1 if filters == prev_filters else 2
    vibration_resnet_model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
vibration_resnet_model.add(keras.layers.GlobalAvgPool2D())
vibration_resnet_model.add(keras.layers.Flatten())
vibration_resnet_model.add(keras.layers.Dense(5,activation="softmax"))
vibration_resnet_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
vibration_resnet_model.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
720/720 [==============================] - 42s 49ms/step - loss: 0.8889 - accuracy: 0.6196 - val_loss: 0.8078 - val_accuracy: 0.5900
Epoch 2/10
720/720 [==============================] - 35s 48ms/step - loss: 0.6824 - accuracy: 0.6649 - val_loss: 0.7198 - val_accuracy: 0.6460
Epoch 3/10
720/720 [==============================] - 35s 48ms/step - loss: 0.6857 - accuracy: 0.6516 - val_loss: 0.6065 - val_accuracy: 0.6904
Epoch 4/10
720/720 [==============================] - 35s 48ms/step - loss: 0.6373 - accuracy: 0.6797 - val_loss: 0.7065 - val_accuracy: 0.6105
Epoch 5/10
720/720 [==============================] - 35s 48ms/step - loss: 0.6557 - accuracy: 0.6713 - val_loss: 0.6224 - val_accuracy: 0.6909
Epoch 6/10
720/720 [==============================] - 35s 48ms/step - loss: 0.6286 - accuracy: 0.6893 - val_loss: 0.5593 - val_accuracy: 0.7191
Epoch 7/10
720/720 [==============================] - 35s 48ms/step - loss: 0.5957 - accuracy: 0.7023 - val_loss: 0.5423 - val_accuracy:

# 3. 모델 예측

In [ ]:
TEST_PATH = '/content/drive/MyDrive/Colab Notebooks/testSet'

In [ ]:
import csv

In [ ]:
def read_test_dir(path):
    if os.path.isdir(path):
        dir_list = os.listdir(path)
    dir_list = sorted(dir_list)
    
    result_f = os.path.join(path,dir_list[-1])

    tmp_dir_list = []
    with open(result_f,'r') as rf:
        tt = csv.reader(rf)
        for i in tt:
            tmp_dir_list.append(i)
    
    # print(len(tmp_dir_list))

    tmp_dir_list = tmp_dir_list[1:]
    test_dir_list = []
    category = []
    kw = []
    for t in tmp_dir_list:
        num = t[0]
        if len(num)<2:
            num = '00'+num
        elif len(num)<3:
            num = '0'+num
        f_name = num+'.csv'
        p = os.path.join(path,f_name)
        test_dir_list.append(p)
        category.append(t[1])
        kw.append(t[2])

    return test_dir_list, category, kw

In [ ]:
def test_predict():
    t,c,k = read_test_dir(TEST_PATH)
    y = {}
    X = None
    for i in range(len(t)):
        f_name = os.path.basename(t[i])
        if f_name == '142.csv':
            print('{} pass : Current라고 되어있으나 3 col이 아닌 1col data, 주최측은 Current가 맞다고 답변'.format(f_name))
            pass
        elif c[i] == 'Current':
            X = pd.read_csv(t[i])
            X.drop('Seq',axis=1,inplace=True)
            X['Motor'] = float(k[i])/100
            
            r1 = cnn_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            r2 = current_resnet_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            print('{}, {}\n cnn : {}\n resnet : {}'.format(f_name,c[i],r1,r2))
            y[f_name] = [r1,r2]
        elif c[i] == 'Kimm':
            X = pd.read_csv(t[i])
            X.drop('Seq',axis=1,inplace=True)
            X['Motor'] = float(k[i])/100

            r1 = kimm_resnet_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            print('{}, {}\n resnet : {}'.format(f_name,c[i],r1))
            y[f_name] = [r1]
        else: # Vibration
            X = pd.read_csv(t[i])
            X.drop('Seq',axis=1,inplace=True)
            X['Motor'] = float(k[i])/100

            r1 = vibration_resnet_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            print('{}, {}\n resnet : {}'.format(f_name,c[i],r1))
            y[f_name] = [r1]
    return y

In [ ]:
test_y_dict = test_predict()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


001.csv, Kimm
 resnet : [3]
002.csv, Kimm
 resnet : [3]
003.csv, Vibration
 resnet : [4]
004.csv, Current
 cnn : [1]
 resnet : [0]
005.csv, Vibration
 resnet : [1]
006.csv, Current
 cnn : [3]
 resnet : [3]
007.csv, Kimm
 resnet : [3]
008.csv, Vibration
 resnet : [1]
009.csv, Kimm
 resnet : [3]
010.csv, Vibration
 resnet : [4]
011.csv, Kimm
 resnet : [1]
012.csv, Vibration
 resnet : [1]
013.csv, Current
 cnn : [3]
 resnet : [3]
014.csv, Vibration
 resnet : [2]
015.csv, Vibration
 resnet : [4]
016.csv, Kimm
 resnet : [4]
017.csv, Current
 cnn : [2]
 resnet : [0]
018.csv, Kimm
 resnet : [4]
019.csv, Vibration
 resnet : [1]
020.csv, Kimm
 resnet : [3]
021.csv, Vibration
 resnet : [1]
022.csv, Current
 cnn : [0]
 resnet : [0]
023.csv, Kimm
 resnet : [3]
024.csv, Kimm
 resnet : [0]
025.csv, Vibration
 resnet : [1]
026.csv, Kimm
 resnet : [0]
027.csv, Vibration
 resnet : [1]
028.csv, Vibration
 resnet : [4]
029.csv, Kimm
 resnet : [1]
030.csv, Vibration
 resnet : [4]
031.csv, Current
 cnn : [